# Hiding Images in Plain Sight: Deep Steganography  Compatible with TensorFlow 2.17 

#### An Unofficial Tensorflow Implementation

In [9]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        os.path.join(dirname, filename)
        #print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
%pylab inline

In [14]:
import tensorflow as tf
print(tf.__version__)


2.19.0


## Imports

In [15]:
import glob
import os
from PIL import Image,ImageOps
import random
import tensorflow as tf
import time
from datetime import datetime
from os.path import join

## Configuration
All Configuration related information is represented in CAPS

In [88]:
TRAIN_PATH = '/kaggle/input/steganography-imagenet/steganography_dataset_imagenet/train'
TRAIN_PATH_COVER='/kaggle/input/steganography-imagenet/steganography_dataset_imagenet/train/cover'
TRAIN_PATH_SECRET='/kaggle/input/steganography-imagenet/steganography_dataset_imagenet/train/secret'
LOGS_Path = "/kaggle/working/logs"
CHECKPOINTS_PATH = '/kaggle/working/checkpoints'


BATCH_SIZE = 8
LEARNING_RATE = .0001
BETA = .75

EXP_NAME = f"beta_{BETA}"

## Helper Methods to Handle images 
The images are first converted to float values between 0 and 1. 

Then they are normalised using the Mean and STD from ImageNet. 

To convert these normalised values back to images, a helper function to undo this normalisation is also written.

In [103]:
files_list_cover = glob.glob(join(TRAIN_PATH_COVER,"**/*"))
files_list_secret= glob.glob(join(TRAIN_PATH_SECRET,"**/*"))

files_list_cover=os.listdir(TRAIN_PATH_COVER)[:500]
files_list_secret=os.listdir(TRAIN_PATH_SECRET)[:500]


#for using images stored in subdirectories in a folder
files_list_cover = glob.glob(join(TRAIN_PATH_COVER, "**", "*"), recursive=True)[:500]  # Recursive
files_list_secret = glob.glob(join(TRAIN_PATH_SECRET, "**", "*"), recursive=True)[:500] 

def normalize_batch(imgs):
    return (imgs -  np.array([0.485, 0.456, 0.406])) /np.array([0.229, 0.224, 0.225])
                                                        
def denormalize_batch(imgs,should_clip=True):
    imgs= (imgs * np.array([0.229, 0.224, 0.225])) + np.array([0.485, 0.456, 0.406])
    
    if should_clip:
        imgs= np.clip(imgs,0,1)
    return imgs

def get_img_batch(files_list_cover,files_list_secret,batch_size=32,size=(224,224),should_normalise=True):
   
    batch_cover = []
    batch_secret = []

    for i in range(batch_size):
        img_secret_path = random.choice(files_list_secret[:500])
        img_cover_path = random.choice(files_list_cover[:500])
        
        img_secret = Image.open(img_secret_path).convert("RGB")
        img_cover = Image.open(img_cover_path).convert("RGB")

        img_secret = np.array(ImageOps.fit(img_secret,size),dtype=np.float32)
        img_cover = np.array(ImageOps.fit(img_cover,size),dtype=np.float32)
        
        img_secret /= 255.
        img_cover /= 255.
        
        batch_cover.append(img_cover)
        batch_secret.append(img_secret)
        
    batch_cover,batch_secret = np.array(batch_cover) , np.array(batch_secret)
    
    if should_normalise:
        batch_cover = normalize_batch(batch_cover)
        batch_secret = normalize_batch(batch_secret)

    return batch_cover,batch_secret
    

## Network Definitions
The three networks are identical in terms of structure. 

1. The Prepare network takes in the **Secret Image** and outputs a (BATCH_SIZE,INPUT_HEIGHT,INPUT_WEIGHT,150) tensor. 

2. The Cover network takes in the output from 1. , and a *Cover Image*. It concatenates these two tensors , giving a (BATCH_SIZE,INPUT_HEIGHT,INPUT_WEIGHT,153) tensor. Then it performs Convolutions , and outputs a (BATCH_SIZE,INPUT_HEIGHT,INPUT_WEIGHT,3) image.

3. The Reveal Network Takes in the output image from Cover Network , and outputs the Revealed Image (which is supposed to look like the **Secret Image**


In [138]:
import tensorflow as tf

def get_prep_network_op(secret_tensor):
    
    with tf.compat.v1.variable_scope('prep_net'):
        
        with tf.compat.v1.variable_scope("3x3_conv_branch"):
            conv_3x3 = tf.keras.layers.Conv2D(filters=50, kernel_size=3, padding='same', activation=tf.nn.relu, name="1")(secret_tensor)
            conv_3x3 = tf.keras.layers.Conv2D(filters=50, kernel_size=3, padding='same', activation=tf.nn.relu, name="2")(conv_3x3)
            conv_3x3 = tf.keras.layers.Conv2D(filters=50, kernel_size=3, padding='same', activation=tf.nn.relu, name="3")(conv_3x3)
            conv_3x3 = tf.keras.layers.Conv2D(filters=50, kernel_size=3, padding='same', activation=tf.nn.relu, name="4")(conv_3x3)
            
        with tf.compat.v1.variable_scope("4x4_conv_branch"):
            conv_4x4 = tf.keras.layers.Conv2D(filters=50, kernel_size=4, padding='same', activation=tf.nn.relu, name="1")(secret_tensor)
            conv_4x4 = tf.keras.layers.Conv2D(filters=50, kernel_size=4, padding='same', activation=tf.nn.relu, name="2")(conv_4x4)
            conv_4x4 = tf.keras.layers.Conv2D(filters=50, kernel_size=4, padding='same', activation=tf.nn.relu, name="3")(conv_4x4)
            conv_4x4 = tf.keras.layers.Conv2D(filters=50, kernel_size=4, padding='same', activation=tf.nn.relu, name="4")(conv_4x4)

        with tf.compat.v1.variable_scope("5x5_conv_branch"):
            conv_5x5 = tf.keras.layers.Conv2D(filters=50, kernel_size=5, padding='same', activation=tf.nn.relu, name="1")(secret_tensor)
            conv_5x5 = tf.keras.layers.Conv2D(filters=50, kernel_size=5, padding='same', activation=tf.nn.relu, name="2")(conv_5x5)
            conv_5x5 = tf.keras.layers.Conv2D(filters=50, kernel_size=5, padding='same', activation=tf.nn.relu, name="3")(conv_5x5)
            conv_5x5 = tf.keras.layers.Conv2D(filters=50, kernel_size=5, padding='same', activation=tf.nn.relu, name="4")(conv_5x5)
            
        concat_1 = tf.keras.layers.Concatenate(axis=3, name='concat_1')([conv_3x3, conv_4x4, conv_5x5])
        
        conv_5x5 = tf.keras.layers.Conv2D(filters=50, kernel_size=5, padding='same', activation=tf.nn.relu, name="final_5x5")(concat_1)
        conv_4x4 = tf.keras.layers.Conv2D(filters=50, kernel_size=4, padding='same', activation=tf.nn.relu, name="final_4x4")(concat_1)
        conv_3x3 = tf.keras.layers.Conv2D(filters=50, kernel_size=3, padding='same', activation=tf.nn.relu, name="final_3x3")(concat_1)
        
        concat_final = tf.keras.layers.Concatenate(axis=3, name='concat_final')([conv_5x5, conv_4x4, conv_3x3])

        return concat_final


    
import tensorflow as tf

def get_hiding_network_op(cover_tensor, prep_output):
    with tf.compat.v1.variable_scope('hide_net'):
        concat_input = tf.concat([cover_tensor, prep_output], axis=3, name='images_features_concat')

        with tf.compat.v1.variable_scope("3x3_conv_branch"):
            conv_3x3 = tf.keras.layers.Conv2D(filters=50, kernel_size=3, padding='same', activation='relu', name="1")(concat_input)
            conv_3x3 = tf.keras.layers.Conv2D(filters=50, kernel_size=3, padding='same', activation='relu', name="2")(conv_3x3)
            conv_3x3 = tf.keras.layers.Conv2D(filters=50, kernel_size=3, padding='same', activation='relu', name="3")(conv_3x3)
            conv_3x3 = tf.keras.layers.Conv2D(filters=50, kernel_size=3, padding='same', activation='relu', name="4")(conv_3x3)

        with tf.compat.v1.variable_scope("4x4_conv_branch"):
            conv_4x4 = tf.keras.layers.Conv2D(filters=50, kernel_size=4, padding='same', activation='relu', name="1")(concat_input)
            conv_4x4 = tf.keras.layers.Conv2D(filters=50, kernel_size=4, padding='same', activation='relu', name="2")(conv_4x4)
            conv_4x4 = tf.keras.layers.Conv2D(filters=50, kernel_size=4, padding='same', activation='relu', name="3")(conv_4x4)
            conv_4x4 = tf.keras.layers.Conv2D(filters=50, kernel_size=4, padding='same', activation='relu', name="4")(conv_4x4)

        with tf.compat.v1.variable_scope("5x5_conv_branch"):
            conv_5x5 = tf.keras.layers.Conv2D(filters=50, kernel_size=5, padding='same', activation='relu', name="1")(concat_input)
            conv_5x5 = tf.keras.layers.Conv2D(filters=50, kernel_size=5, padding='same', activation='relu', name="2")(conv_5x5)
            conv_5x5 = tf.keras.layers.Conv2D(filters=50, kernel_size=5, padding='same', activation='relu', name="3")(conv_5x5)
            conv_5x5 = tf.keras.layers.Conv2D(filters=50, kernel_size=5, padding='same', activation='relu', name="4")(conv_5x5)

        concat_1 = tf.concat([conv_3x3, conv_4x4, conv_5x5], axis=3, name='concat_1')

        conv_5x5 = tf.keras.layers.Conv2D(filters=50, kernel_size=5, padding='same', activation='relu', name="final_5x5")(concat_1)
        conv_4x4 = tf.keras.layers.Conv2D(filters=50, kernel_size=4, padding='same', activation='relu', name="final_4x4")(concat_1)
        conv_3x3 = tf.keras.layers.Conv2D(filters=50, kernel_size=3, padding='same', activation='relu', name="final_3x3")(concat_1)

        concat_final = tf.concat([conv_5x5, conv_4x4, conv_3x3], axis=3, name='concat_final')
        output = tf.keras.layers.Conv2D(filters=3, kernel_size=1, padding='same', name='output')(concat_final)

        return output

        
        
import tensorflow as tf

def get_reveal_network_op(container_tensor):
    with tf.compat.v1.variable_scope('reveal_net'):
        
        with tf.compat.v1.variable_scope("3x3_conv_branch"):
            conv_3x3 = tf.keras.layers.Conv2D(filters=50, kernel_size=3, padding='same', activation='relu', name="1")(container_tensor)
            conv_3x3 = tf.keras.layers.Conv2D(filters=50, kernel_size=3, padding='same', activation='relu', name="2")(conv_3x3)
            conv_3x3 = tf.keras.layers.Conv2D(filters=50, kernel_size=3, padding='same', activation='relu', name="3")(conv_3x3)
            conv_3x3 = tf.keras.layers.Conv2D(filters=50, kernel_size=3, padding='same', activation='relu', name="4")(conv_3x3)
        
        with tf.compat.v1.variable_scope("4x4_conv_branch"):
            conv_4x4 = tf.keras.layers.Conv2D(filters=50, kernel_size=4, padding='same', activation='relu', name="1")(container_tensor)
            conv_4x4 = tf.keras.layers.Conv2D(filters=50, kernel_size=4, padding='same', activation='relu', name="2")(conv_4x4)
            conv_4x4 = tf.keras.layers.Conv2D(filters=50, kernel_size=4, padding='same', activation='relu', name="3")(conv_4x4)
            conv_4x4 = tf.keras.layers.Conv2D(filters=50, kernel_size=4, padding='same', activation='relu', name="4")(conv_4x4)

        with tf.compat.v1.variable_scope("5x5_conv_branch"):
            conv_5x5 = tf.keras.layers.Conv2D(filters=50, kernel_size=5, padding='same', activation='relu', name="1")(container_tensor)
            conv_5x5 = tf.keras.layers.Conv2D(filters=50, kernel_size=5, padding='same', activation='relu', name="2")(conv_5x5)
            conv_5x5 = tf.keras.layers.Conv2D(filters=50, kernel_size=5, padding='same', activation='relu', name="3")(conv_5x5)
            conv_5x5 = tf.keras.layers.Conv2D(filters=50, kernel_size=5, padding='same', activation='relu', name="4")(conv_5x5)

        concat_1 = tf.concat([conv_3x3, conv_4x4, conv_5x5], axis=3, name='concat_1')

        conv_5x5 = tf.keras.layers.Conv2D(filters=50, kernel_size=5, padding='same', activation='relu', name="final_5x5")(concat_1)
        conv_4x4 = tf.keras.layers.Conv2D(filters=50, kernel_size=4, padding='same', activation='relu', name="final_4x4")(concat_1)
        conv_3x3 = tf.keras.layers.Conv2D(filters=50, kernel_size=3, padding='same', activation='relu', name="final_3x3")(concat_1)

        concat_final = tf.concat([conv_5x5, conv_4x4, conv_3x3], axis=3, name='concat_final')

        output = tf.keras.layers.Conv2D(filters=3, kernel_size=1, padding='same', name='output')(concat_final)

    return output


def get_noise_layer_op(tensor,std=.1):
    with tf.compat.v1.variable_scope("noise_layer"):
        return tensor + tf.random.normal(shape=tf.shape(tensor), mean=0.0, stddev=std, dtype=tf.float32) 
    
def get_loss_op(secret_true,secret_pred,cover_true,cover_pred,beta=.5):
    with tf.compat.v1.variable_scope("losses"):
        beta = tf.constant(beta,name="beta")
        secret_mse = tf.keras.losses.MeanSquaredError()(secret_true, secret_pred)
        cover_mse = tf.keras.losses.MeanSquaredError()(cover_true, cover_pred)
        final_loss = cover_mse + beta*secret_mse
        return final_loss , secret_mse , cover_mse 

def get_tensor_to_img_op(tensor):
    with tf.compat.v1.variable_scope("",reuse=True):
        t = tensor*tf.convert_to_tensor([0.229, 0.224, 0.225]) + tf.convert_to_tensor([0.485, 0.456, 0.406])
        return tf.clip_by_value(t,0,1)

In [187]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()  # Disable eager execution (needed for TensorFlow 1.x compatibility)

def prepare_training_graph(secret_tensor,cover_tensor,global_step_tensor):

    prep_output_op = get_prep_network_op(secret_tensor)
    hiding_output_op = get_hiding_network_op(cover_tensor=cover_tensor,prep_output=prep_output_op)
    noise_add_op = get_noise_layer_op(hiding_output_op)
    reveal_output_op = get_reveal_network_op(noise_add_op)
    
    loss_op,secret_loss_op,cover_loss_op = get_loss_op(secret_tensor,reveal_output_op,cover_tensor,hiding_output_op,beta=BETA)

    minimize_op = tf.compat.v1.train.AdamOptimizer(LEARNING_RATE).minimize(loss_op,global_step=global_step_tensor)
    
    tf.compat.v1.summary.scalar('loss', loss_op)
    tf.compat.v1.summary.scalar('reveal_net_loss', secret_loss_op)
    tf.compat.v1.summary.scalar('cover_net_loss', cover_loss_op)

    tf.compat.v1.summary.image('secret',get_tensor_to_img_op(secret_tensor),max_outputs=1)
    tf.compat.v1.summary.image('cover',get_tensor_to_img_op(cover_tensor),max_outputs=1)
    tf.compat.v1.summary.image('hidden',get_tensor_to_img_op(hiding_output_op),max_outputs=1)
    tf.compat.v1.summary.image('hidden_noisy',get_tensor_to_img_op(noise_add_op),max_outputs=1)
    tf.compat.v1.summary.image('revealed',get_tensor_to_img_op(reveal_output_op),max_outputs=1)

    merged_summary_op = tf.compat.v1.summary.merge_all()
    
    return minimize_op, merged_summary_op 

In [191]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()  # Disable eager execution (needed for TensorFlow 1.x compatibility)

def prepare_test_graph(secret_tensor, cover_tensor):
    with tf.compat.v1.variable_scope("", reuse=tf.compat.v1.AUTO_REUSE):
        prep_output_op = get_prep_network_op(secret_tensor)
        hiding_output_op = get_hiding_network_op(cover_tensor=cover_tensor, prep_output=prep_output_op)
        reveal_output_op = get_reveal_network_op(hiding_output_op)

        loss_op, secret_loss_op, cover_loss_op = get_loss_op(secret_tensor, reveal_output_op, cover_tensor, hiding_output_op)

        tf.compat.v1.summary.scalar('loss', loss_op)
        tf.compat.v1.summary.scalar('reveal_net_loss', secret_loss_op)
        tf.compat.v1.summary.scalar('cover_net_loss', cover_loss_op)


        tf.compat.v1.summary.image('secret', get_tensor_to_img_op(secret_tensor), max_outputs=1)
        tf.compat.v1.summary.image('cover', get_tensor_to_img_op(cover_tensor), max_outputs=1)
        tf.compat.v1.summary.image('hidden', get_tensor_to_img_op(hiding_output_op), max_outputs=1)
        tf.compat.v1.summary.image('revealed', get_tensor_to_img_op(reveal_output_op), max_outputs=1)

        merged_summary_op = tf.compat.v1.summary.merge_all()

        return merged_summary_op

In [133]:
def prepare_deployment_graph(secret_tensor,cover_tensor,covered_tensor):
    with tf.compat.v1.variable_scope("",reuse=True):

        prep_output_op = get_prep_network_op(secret_tensor)
        hiding_output_op = get_hiding_network_op(cover_tensor=cover_tensor,prep_output=prep_output_op)

        reveal_output_op = get_reveal_network_op(covered_tensor)

        return hiding_output_op ,  reveal_output_op

In [134]:
import tensorflow as tf

# Enable TensorFlow 1.x compatibility mode
tf.compat.v1.disable_eager_execution()

In [174]:
import tensorflow as tf

# Explicitly enable eager execution (which is the default in TF 2.x)
tf.compat.v1.disable_eager_execution()
tf.compat.v1.reset_default_graph()

# Use tf.compat.v1.Session() instead
sess = tf.compat.v1.Session(graph=tf.Graph())


In [172]:
#sess = tf.InteractiveSession(graph=tf.Graph())

In [175]:
import tensorflow as tf

# Disable eager execution (required for placeholder-based execution)
tf.compat.v1.disable_eager_execution()

# Define placeholders using TensorFlow 1.x compatibility mode
secret_tensor = tf.compat.v1.placeholder(shape=[None, 224, 224, 3], dtype=tf.float32, name="input_prep")
cover_tensor = tf.compat.v1.placeholder(shape=[None, 224, 224, 3], dtype=tf.float32, name="input_hide")
global_step_tensor = tf.compat.v1.Variable(0, trainable=False, name='global_step')

# Ensure that all other TensorFlow 1.x functions use tf.compat.v1
writer = tf.compat.v1.summary.FileWriter(join(LOGS_Path, EXP_NAME))

In [192]:

test_op = prepare_test_graph(secret_tensor,cover_tensor)

covered_tensor = tf.compat.v1.placeholder(shape=[None,224,224,3],dtype=tf.float32,name="deploy_covered")
deploy_hide_image_op , deploy_reveal_image_op = prepare_deployment_graph(secret_tensor,cover_tensor,covered_tensor)

In [193]:
train_op , summary_op = prepare_training_graph(secret_tensor,cover_tensor,global_step_tensor)

In [178]:
#saver = tf.compat.v1.train.Saver(max_to_keep=1)
#sess.run(tf.compat.v1.global_variables_initializer())
# saver.restore(sess,join(CHECKPOINTS_PATH,EXP_NAME))

In [179]:
total_steps = len(files_list_cover)//BATCH_SIZE + 1

In [194]:
from tqdm import tqdm

with tf.compat.v1.Session() as sess:
    sess.run(tf.compat.v1.global_variables_initializer())  # Ensure variables are initialized

    for ep in range(100):

        with tqdm(range(total_steps), desc=f"Epoch {ep+1}/100", unit="step") as pbar:
          for step in pbar:  # Use tqdm progress bar

            covers,secrets = get_img_batch(files_list_cover=files_list_cover, files_list_secret=files_list_secret,batch_size=BATCH_SIZE)
            print(f"cover shape is {covers.shape} and secret shape is {secrets.shape}")
            sess.run([train_op],feed_dict={"input_prep:0":secrets,"input_hide:0":covers})
            
            if step % 10 ==0 :
                
                summary,global_step = sess.run([summary_op,global_step_tensor],feed_dict={"input_prep:0":secrets,"input_hide:0":covers})
                writer.add_summary(summary,global_step)
                
            if step % 100 ==0 :
                
                covers,secrets = get_img_batch(files_list_cover=files_list_cover, files_list_secret=files_list_secret,batch_size=1)
                summary,global_step = sess.run([test_op,global_step_tensor],feed_dict={"input_prep:0":secrets,"input_hide:0":covers})
                writer.add_summary(summary,global_step)
    
        
        save_path = saver.save(sess, join(CHECKPOINTS_PATH,EXP_NAME+".chkp"),global_step=global_step)


Epoch 1/100:   0%|          | 0/63 [00:00<?, ?step/s]

cover shape is (8, 224, 224, 3) and secret shape is (8, 224, 224, 3)


Epoch 1/100:   2%|▏         | 1/63 [03:38<3:45:23, 218.12s/step]

cover shape is (8, 224, 224, 3) and secret shape is (8, 224, 224, 3)


Epoch 1/100:   3%|▎         | 2/63 [05:21<2:33:01, 150.52s/step]

...

cover shape is (8, 224, 224, 3) and secret shape is (8, 224, 224, 3)


Epoch 1/100:   6%|▋         | 4/63 [08:45<1:56:10, 118.15s/step]

cover shape is (8, 224, 224, 3) and secret shape is (8, 224, 224, 3)


Epoch 1/100:   6%|▋         | 4/63 [10:28<2:34:28, 157.10s/step]


KeyboardInterrupt: 

In [ ]:
# sess.close()

writer.close()

In [ ]:
covers,secrets = get_img_batch(files_list_cover=files_list_cover,files_list_secret=files_list_secret,batch_size=1)

cover = covers.squeeze()
secret = secrets.squeeze()
# Plot and save the cover image
plt.figure()
plt.imshow(denormalize_batch(cover))
plt.title("Cover")
plt.savefig("cover.png")   # Saves the figure to cover.png
plt.show()

# Plot and save the secret image
plt.figure()
plt.imshow(denormalize_batch(secret))
plt.title("Secret")
plt.savefig("secret.png")  # Saves the figure to secret.png
plt.show()

In [ ]:
# Run the model to get the hidden image
hidden = sess.run(deploy_hide_image_op, feed_dict={'input_prep:0': secrets, 'input_hide:0': covers})

plt.figure()
plt.imshow(denormalize_batch(hidden.squeeze()))
plt.title("Hidden")
plt.savefig("hidden.png")
plt.show()

# Run the model to get the revealed image
revealed = sess.run(deploy_reveal_image_op, feed_dict={'deploy_covered:0': hidden})

plt.figure()
plt.imshow(denormalize_batch(revealed.squeeze()))
plt.title("Revealed")
plt.savefig("revealed.png")
plt.show()

# Optionally, save the output of hiding network if available:
plt.figure()
plt.imshow(np.clip(hiding_output.squeeze(), 0, 1))
plt.title("Hiding Output")
plt.savefig("hiding_output.png")
plt.show()

In [ ]:
hidden = sess.run(deploy_hide_image_op,feed_dict={'input_prep:0':secrets,'input_hide:0':covers})

plt.imshow(denormalize_batch(hidden.squeeze()))

In [ ]:
revealed = sess.run(deploy_reveal_image_op,feed_dict={'deploy_covered:0':hidden})

plt.imshow(denormalize_batch(revealed.squeeze()))

In [ ]:
plt.imshow(np.clip(hiding_output.squeeze(),0,1))

In [ ]:
hiding_network_output = sess.run([hiding_output_op],
                                  feed_dict={secret_tensor:secrets,cover_tensor:covers})[0]

In [ ]:
# plt.imshow(np.clip(hiding_network_output[0],0,1))

In [ ]:
# join("OK",".OK",".OK")

In [ ]:
# covers,secrets = get_img_batch(files_list=files_list,batch_size=BATCH_SIZE)

# type(secrets)

In [ ]:
# files_list

In [ ]:
# image_str = tf.placeholder(tf.string)
# im_tf = tf.image.decode_image(image_str)

In [ ]:
# cover_imgs = []
# hidden_imgs = []
# hidden_noisy = []
# reveal_imgs = []
# secret_imgs = []


# count = 0
# for e in tf.train.summary_iterator(join(LOGS_Path,'beta_0.25','events.out.tfevents.1516061354.pcvirus')):
#     for v in e.summary.value:
#         if v.tag == 'train/train/cover/image':
#             output = im_tf.eval(feed_dict={image_str:v.image.encoded_image_string})
#             cover_imgs.append(output)
            
#         if v.tag == 'train/train/hidden/image':
#             output = im_tf.eval(feed_dict={image_str:v.image.encoded_image_string})
#             hidden_imgs.append(output)
        
#         if v.tag == 'train/train/hidden_noisy/image':
#             output = im_tf.eval(feed_dict={image_str:v.image.encoded_image_string})
#             hidden_noisy.append(output)
            
#         if v.tag == 'train/train/revealed/image':
#             output = im_tf.eval(feed_dict={image_str:v.image.encoded_image_string})
#             reveal_imgs.append(output)
            
#         if v.tag == 'train/train/secret/image':
#             output = im_tf.eval(feed_dict={image_str:v.image.encoded_image_string})
#             secret_imgs.append(output)
        